In [ ]:
import pandas as pd
from BayesABTest import ab_test_model
from matplotlib import pyplot as plt
import numpy as np
import json

# load inputs
experiment_file = open('experiment.json')
experiment = json.load(experiment_file)

In [ ]:
# generate binomial test data
# THIS CELL SHOULD BE REMOVED IN PRODUCTION
# data needs to be in a dataframe format where one column has the variant names, and one is a binomial outcome variable
# expected column names are "bucket" for the variants. The binary outcome column name is up to you

c_binom = np.random.binomial(size=100000, p=.1, n=1)
t_binom = np.random.binomial(size=100000, p=.1025, n=1)

c_binom_df = pd.DataFrame(c_binom, columns = ['outcomes'])
t_binom_df = pd.DataFrame(t_binom, columns = ['outcomes'])

c_binom_df['bucket'] = 'control'
t_binom_df['bucket'] = 'test'

variants_df = c_binom_df.append(t_binom_df)

# load and set required inputs:

confidence_interval = .9
outcome_col = 'outcomes'
control_variant = experiment['control_variant_name']

# rename
df = variants_df

In [ ]:
# # for production, use this to pull data
# # ideally there is a table that holds all experimentation data with outcomes at the desired level of granularity (eg user)
# # this would make it possible to templatize data sourcing for this kind of analysis
# # another possibility would be to have multiple queries (stored elsewhere) that shape data as desired for the specific experiment or KPI being measured

# confidence_interval = .9
# outcome_col = 'outcomes' # The binary outcome column name is up to you, but needs to be specified
# control_variant = experiment['control_variant_name']

# df = pd.read_csv('experiment_data.csv')

In [ ]:
%%capture
# initialize class with the data and some optional variables
first_test = ab_test_model(df, 
                           metric=outcome_col, 
                           prior_info='uninformed', 
                           prior_function='beta', 
                           control_bucket_name=control_variant,
                           confidence_level=1-confidence_interval,
                           debug=True)

# run public methods
first_test.fit()

## TLDR:

In [ ]:
print(f"Observations: {len(df['bucket'])}")

In [ ]:
variants = df['bucket'].unique().tolist()
variants.remove(control_variant)

lower_bound = (1-confidence_interval)/2*100
upper_bound = (confidence_interval + (1-confidence_interval)/2) * 100

for var in variants:
    lift = np.percentile(first_test.lift[var]['control'], 50)
    likelihood_of_improvement = round(sum(i > 0 for i in first_test.lift[var][control_variant]) / len(first_test.lift[var][control_variant])*100,2)

    if not np.percentile(first_test.lift[var][control_variant], lower_bound*5) <= 0 <= np.percentile(first_test.lift[var][control_variant], upper_bound) and likelihood_of_improvement >= 90.00:
        print(f"The {var} variant is better!")
        print(str(f"Probable lift: {round(lift*100,2)}%"))
        print(f"Likelihood of Improvement: {likelihood_of_improvement}%")
    else:
        print(f"No improvement for {var} variant, keep test running or call it")

In [ ]:
first_test.plot()
plt.show()